#  Spam SMS Detection using Machine Learning

This project uses Natural Language Processing (NLP) techniques and a Naive Bayes classifier to detect whether an SMS message is spam or legitimate (ham). The dataset consists of thousands of real SMS messages labeled as spam or ham.

### Libraries Used:
- pandas
- scikit-learn
- nltk


In [3]:
# Drop the unnamed extra columns
df = df[['v1', 'v2']]

# Rename columns to something meaningful
df.columns = ['label', 'message']

# Show the cleaned data
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import pandas as pd

# Load the dataset (note: folder name is case-sensitive on some systems)
df = pd.read_csv("Dataset/spam.csv", encoding='latin-1')

# Show first 5 rows
df.head()


In [8]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Download stopwords if not done already
nltk.download('stopwords')

ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df['cleaned_message'] = df['message'].apply(clean_text)

# Display cleaned messages
df[['message', 'cleaned_message']].head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raamaandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,message,cleaned_message
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goe usf live around though


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=3000)

# Transform the cleaned messages
X = tfidf.fit_transform(df['cleaned_message']).toarray()

# Encode 'ham' as 0 and 'spam' as 1
y = df['label'].map({'ham': 0, 'spam': 1})

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.97847533632287

Confusion Matrix:
 [[965   0]
 [ 24 126]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [11]:
def predict_message(msg):
    cleaned = clean_text(msg)
    vectorized = tfidf.transform([cleaned]).toarray()
    prediction = model.predict(vectorized)[0]
    return "Spam" if prediction == 1 else "Ham"

# Try your own
print(predict_message("Congratulations! You’ve won a FREE vacation to Maldives. Call now!"))
print(predict_message("Hey are we meeting tomorrow at 5?"))


Spam
Ham


##  Conclusion

This Spam SMS Detection model achieved **97.8% accuracy** using Naive Bayes and TF-IDF vectorization.  
It effectively distinguishes spam messages from legitimate ones and can be integrated into any SMS filtering system.

- Model: Multinomial Naive Bayes
- Vectorizer: TF-IDF (3000 features)
- Accuracy: 97.8%
